In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, DateTime, Numeric

In [2]:
#Import BTC-USD.csv
btc_file="Resources/BTC-USD.csv"
btc_usd_df=pd.read_csv(btc_file)
btc_usd_df.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,9/17/2014,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,9/18/2014,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,9/19/2014,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,9/20/2014,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,9/21/2014,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0


In [3]:
#Import ETH-USD.csv
eth_file="Resources/ETH-USD.csv"
eth_usd_df=pd.read_csv(eth_file)
eth_usd_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,11/9/2017,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08
1,11/10/2017,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08
2,11/11/2017,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08
3,11/12/2017,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09
4,11/13/2017,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09


In [4]:
# Adding a daily percent change column
btc_percent_change = ((btc_usd_df["Close"]-btc_usd_df["Open"])/btc_usd_df["Open"])*100
btc_usd_df["Daily_Percent_Change"] = btc_percent_change
btc_usd_df=btc_usd_df.round({"Daily_Percent_Change":2})

eth_percent_change = ((eth_usd_df["Close"]-eth_usd_df["Open"])/eth_usd_df["Open"])*100
eth_usd_df["Daily_Percent_Change"] = eth_percent_change
eth_usd_df=eth_usd_df.round({"Daily_Percent_Change":2})

eth_usd_df.head()


,Date,Open,High,Low,Close,Adj Close,Volume,Daily_Percent_Change
0,11/9/2017,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08,3.97
1,11/10/2017,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08,-6.68
2,11/11/2017,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08,5.39
3,11/12/2017,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09,-2.16
4,11/13/2017,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09,3.16


In [5]:
# Renamed BTC columns
btc_rename=btc_usd_df.rename (columns={"Open":"BTC_Open",
                                      "High":"BTC_High",
                                      "Low":"BTC_Low",
                                      "Close":"BTC_Close",
                                      "Adj Close":"BTC_Adj_Close",
                                      "Volume":"BTC_Volume",
                                      "Daily_Percent_Change":"BTC_Daily_Percent_Change"})

btc_rename.head()

,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Adj_Close,BTC_Volume,BTC_Daily_Percent_Change
0,9/17/2014,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0,-1.83
1,9/18/2014,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0,-7.10
2,9/19/2014,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0,-6.91
3,9/20/2014,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0,3.61
4,9/21/2014,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0,-2.27


In [6]:
# Renamed ETH columns
eth_rename=eth_usd_df.rename (columns={"Open":"ETH_Open",
                                      "High":"ETH_High",
                                      "Low":"ETH_Low",
                                      "Close":"ETH_Close",
                                      "Adj Close":"ETH_Adj_Close",
                                      "Volume":"ETH_Volume",
                                      "Daily_Percent_Change":"ETH_Daily_Percent_Change"})

eth_rename.head()

,Date,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Adj_Close,ETH_Volume,ETH_Daily_Percent_Change
0,11/9/2017,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08,3.97
1,11/10/2017,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08,-6.68
2,11/11/2017,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08,5.39
3,11/12/2017,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09,-2.16
4,11/13/2017,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09,3.16


In [7]:
# Setting the Date column as index
# btc_final_df=btc_rename.set_index(["Date"])
# btc_final_df.head()

In [8]:
# Setting the Date column as index
# eth_final_df=eth_rename.set_index(["Date"])
# eth_final_df.head()

In [10]:
#Joined the Dataframes on the Date
combined_df=btc_rename.merge(eth_rename,how="inner", on="Date")
combined_df

,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Adj_Close,BTC_Volume,BTC_Daily_Percent_Change,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Adj_Close,ETH_Volume,ETH_Daily_Percent_Change
0,11/9/2017,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3.226250e+09,-4.07,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08,3.97
1,11/10/2017,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5.208250e+09,-7.74,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08,-6.68
2,11/11/2017,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4.908680e+09,-3.94,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08,5.39
3,11/12/2017,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8.957350e+09,-5.49,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09,-2.16
4,11/13/2017,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6.263250e+09,10.46,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09,3.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1653,5/20/2022,30311.119140,30664.976560,28793.605470,29200.740230,29200.740230,3.074938e+10,-3.66,2018.000122,2054.795898,1926.676758,1961.315674,1961.315674,1.589248e+10,-2.81
1654,5/21/2022,29199.859380,29588.869140,29027.394530,29432.226560,29432.226560,1.727484e+10,0.80,1961.017944,1985.395996,1944.265137,1974.518311,1974.518311,8.546822e+09,0.69
1655,5/22/2022,29432.472660,30425.861330,29275.183590,30323.722660,30323.722660,2.163153e+10,3.03,1974.670654,2047.191406,1966.038818,2043.170166,2043.170166,1.094112e+10,3.47
1656,5/23/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Dropping null values
combined_df = combined_df.dropna()

combined_df

,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Adj_Close,BTC_Volume,BTC_Daily_Percent_Change,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Adj_Close,ETH_Volume,ETH_Daily_Percent_Change
0,11/9/2017,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3.226250e+09,-4.07,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08,3.97
1,11/10/2017,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5.208250e+09,-7.74,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08,-6.68
2,11/11/2017,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4.908680e+09,-3.94,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08,5.39
3,11/12/2017,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8.957350e+09,-5.49,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09,-2.16
4,11/13/2017,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6.263250e+09,10.46,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09,3.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1652,5/19/2022,28720.359380,30430.751950,28708.955080,30314.333980,30314.333980,3.377345e+10,5.55,1916.149536,2028.388184,1907.020508,2018.336182,2018.336182,1.873425e+10,5.33
1653,5/20/2022,30311.119140,30664.976560,28793.605470,29200.740230,29200.740230,3.074938e+10,-3.66,2018.000122,2054.795898,1926.676758,1961.315674,1961.315674,1.589248e+10,-2.81
1654,5/21/2022,29199.859380,29588.869140,29027.394530,29432.226560,29432.226560,1.727484e+10,0.80,1961.017944,1985.395996,1944.265137,1974.518311,1974.518311,8.546822e+09,0.69
1655,5/22/2022,29432.472660,30425.861330,29275.183590,30323.722660,30323.722660,2.163153e+10,3.03,1974.670654,2047.191406,1966.038818,2043.170166,2043.170166,1.094112e+10,3.47


In [13]:
combined_df.dtypes


Date                         object
BTC_Open                    float64
BTC_High                    float64
BTC_Low                     float64
BTC_Close                   float64
BTC_Adj_Close               float64
BTC_Volume                  float64
BTC_Daily_Percent_Change    float64
ETH_Open                    float64
ETH_High                    float64
ETH_Low                     float64
ETH_Close                   float64
ETH_Adj_Close               float64
ETH_Volume                  float64
ETH_Daily_Percent_Change    float64
dtype: object

In [14]:
# Creating a raw combined dataframe by appending all entries

#Adding a column to label the type of cryptocurrency
btc_usd_df["Crypto"] = "BTC"
eth_usd_df["Crypto"] = "ETH"

#Appending the dataframes together
raw_combined_df = btc_usd_df.append(eth_usd_df)

#Converting the Date column to datetime and sorting it ascending
raw_combined_df['Date'] = pd.to_datetime(raw_combined_df['Date'], format='%m/%d/%Y')
raw_combined_df = raw_combined_df.sort_values(by=["Date"])

#Dropping null values
raw_combined_df = raw_combined_df.dropna()

#Reset the index
raw_combined_df = raw_combined_df.reset_index(drop = True)

#Reorganize columns
raw_combined_df = raw_combined_df[["Date","Crypto","Open","High","Low","Close","Adj Close","Volume","Daily_Percent_Change"]]

raw_combined_df

C:\Users\skdab\AppData\Local\Temp\ipykernel_21672\1776269638.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_combined_df = btc_usd_df.append(eth_usd_df)


,Date,Crypto,Open,High,Low,Close,Adj Close,Volume,Daily_Percent_Change
0,2014-09-17,BTC,465.864014,468.174011,452.421997,457.334015,457.334015,2.105680e+07,-1.83
1,2014-09-18,BTC,456.859985,456.859985,413.104004,424.440002,424.440002,3.448320e+07,-7.10
2,2014-09-19,BTC,424.102997,427.834991,384.532013,394.795990,394.795990,3.791970e+07,-6.91
3,2014-09-20,BTC,394.673004,423.295990,389.882996,408.903992,408.903992,3.686360e+07,3.61
4,2014-09-21,BTC,408.084991,412.425995,393.181000,398.821014,398.821014,2.658010e+07,-2.27
...,...,...,...,...,...,...,...,...,...
4458,2022-05-21,BTC,29199.859380,29588.869140,29027.394530,29432.226560,29432.226560,1.727484e+10,0.80
4459,2022-05-22,BTC,29432.472660,30425.861330,29275.183590,30323.722660,30323.722660,2.163153e+10,3.03
4460,2022-05-22,ETH,1974.670654,2047.191406,1966.038818,2043.170166,2043.170166,1.094112e+10,3.47
4461,2022-05-24,BTC,29083.347660,29280.664060,29083.347660,29275.783200,29275.783200,3.177276e+10,0.66


In [15]:
raw_combined_df=raw_combined_df.rename (columns={"Adj Close":"Adj_Close"})

raw_combined_df

,Date,Crypto,Open,High,Low,Close,Adj_Close,Volume,Daily_Percent_Change
0,2014-09-17,BTC,465.864014,468.174011,452.421997,457.334015,457.334015,2.105680e+07,-1.83
1,2014-09-18,BTC,456.859985,456.859985,413.104004,424.440002,424.440002,3.448320e+07,-7.10
2,2014-09-19,BTC,424.102997,427.834991,384.532013,394.795990,394.795990,3.791970e+07,-6.91
3,2014-09-20,BTC,394.673004,423.295990,389.882996,408.903992,408.903992,3.686360e+07,3.61
4,2014-09-21,BTC,408.084991,412.425995,393.181000,398.821014,398.821014,2.658010e+07,-2.27
...,...,...,...,...,...,...,...,...,...
4458,2022-05-21,BTC,29199.859380,29588.869140,29027.394530,29432.226560,29432.226560,1.727484e+10,0.80
4459,2022-05-22,BTC,29432.472660,30425.861330,29275.183590,30323.722660,30323.722660,2.163153e+10,3.03
4460,2022-05-22,ETH,1974.670654,2047.191406,1966.038818,2043.170166,2043.170166,1.094112e+10,3.47
4461,2022-05-24,BTC,29083.347660,29280.664060,29083.347660,29275.783200,29275.783200,3.177276e+10,0.66


In [16]:
raw_combined_df.dtypes


Date                    datetime64[ns]
Crypto                          object
Open                           float64
High                           float64
Low                            float64
Close                          float64
Adj_Close                      float64
Volume                         float64
Daily_Percent_Change           float64
dtype: object

In [27]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Creates Classes which will serve as the anchor points for our Tables
class Crypto(Base):
    __tablename__ = 'Crypto'
    id = Column(Integer, primary_key=True)
    Date = Column(DateTime)
    Crypto = Column(String)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column(Float)
    Volume = Column(Float)
    Daily_Percent_Change = Column(Float)
    
# Creates Classes which will serve as the anchor points for our Tables
class BTC_ETH(Base):
    __tablename__ = 'BTC_ETH'
    id = Column(Integer, primary_key=True)
    Date = Column(DateTime)
    BTC_Open = Column(Float)
    BTC_High = Column(Float)
    BTC_Low = Column(Float)
    BTC_Close = Column(Float)
    BTC_Adj_Close = Column(Float)
    BTC_Volume = Column(Float)
    BTC_Daily_Percent_Change = Column(Float)
    ETH_Open = Column(Float)
    ETH_High = Column(Float)
    ETH_Low = Column(Float)
    ETH_Close = Column(Float)
    ETH_Adj_Close = Column(Float)
    ETH_Volume = Column(Float)
    ETH_Daily_Percent_Change = Column(Float)

In [28]:
# Connect to Local Database
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'crypto_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

# Create tables in database
conn = engine.connect()
Base.metadata.create_all(engine)

In [29]:
# Check for tables
engine.table_names()


C:\Users\skdab\AppData\Local\Temp\ipykernel_21672\650827255.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Crypto', 'BTC_ETH']

In [30]:
raw_combined_df.to_sql(name='Crypto', con=engine, if_exists='append', index=False)

463

In [32]:
combined_df.to_sql(name='BTC_ETH', con=engine, if_exists='append', index=False)

657

In [33]:
pd.read_sql_query('select * from "Crypto"', con=engine).head()

,id,Date,Crypto,Open,High,Low,Close,Adj_Close,Volume,Daily_Percent_Change
0,1,2014-09-17,BTC,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0,-1.83
1,2,2014-09-18,BTC,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0,-7.10
2,3,2014-09-19,BTC,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0,-6.91
3,4,2014-09-20,BTC,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0,3.61
4,5,2014-09-21,BTC,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0,-2.27


In [34]:
pd.read_sql_query('select * from "BTC_ETH"', con=engine).head()

,Date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Adj_Close,BTC_Volume,BTC_Daily_Percent_Change,ETH_Open,ETH_High,ETH_Low,ETH_Close,ETH_Adj_Close,ETH_Volume,ETH_Daily_Percent_Change
0,11/9/2017,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3.226250e+09,-4.07,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08,3.97
1,11/10/2017,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5.208250e+09,-7.74,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08,-6.68
2,11/11/2017,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4.908680e+09,-3.94,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08,5.39
3,11/12/2017,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8.957350e+09,-5.49,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09,-2.16
4,11/13/2017,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6.263250e+09,10.46,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09,3.16
